In [16]:
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pymc3 as pm
import pandas as pd
import theano
import arviz as az

from scipy import stats
import xarray as xr

import itertools
from theano import tensor as tt

plt.style.use('bmh')
plt.rcParams["figure.figsize"] = (8,5)
#WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.

In [17]:
data = pd.read_csv("./IMA_recommendation_simulation_data.csv")
data = data.dropna()

In [18]:
def zz(value):
    if value < 100:
        return 'Z0'
    elif value < 200:
        return 'Z1'
    elif value < 300:
        return 'Z2'
    elif value < 400:
        return 'Z3'
    elif value < 500:
        return 'Z4'
    elif value < 600:
        return 'Z5'
    elif value < 700:
        return 'Z6'
    elif value < 800:
        return 'Z7'
    elif value < 900:
        return 'Z8'
    elif value < 1000:
        return 'Z9'
    else:
        return 'Error'


data['ZipZone_origin'] = data.apply(lambda column: zz(column['origin_dat_ref']),axis=1)
data['ZipZone_dest'] = data.apply(lambda column: zz(column['dest_dat_ref']),axis=1)

In [19]:
data['ZipZone_pair'] = data.apply(lambda column: (column['ZipZone_origin'], column['ZipZone_dest']), axis=1)
data.head()

,Unnamed: 0,request_id,week_id,weekday,miles,order_equipment_type,order_distance,order_num_stops,order_origin_weight,lead_days,color,origin_dat_ref,dest_dat_ref,rate_norm,est_cost_norm,CurrentCondition,ZipZone_origin,ZipZone_dest,ZipZone_pair
0,0,c2653eda11cd4d94879d1946392eb2b5,4,Sat,1856,V,1848.0,2,40000.0,3,RED,633,978,2.273442,2.138631,Accepted,Z6,Z9,"(Z6, Z9)"
1,1,65e3dcc84a2246e68ea8636a967b31b2,3,Mon,56,V,47.0,2,12729.0,3,RED,604,604,-0.971030,-1.017024,Rejected,Z6,Z6,"(Z6, Z6)"
2,2,0a86f005640a4204b46c95ca033a7ced,1,Fri,259,V,255.0,2,44176.0,3,RED,761,770,-0.393352,-0.475805,Accepted,Z7,Z7,"(Z7, Z7)"
3,3,1105b4bc7b444f7985d4ad810e7acbc2,3,Thu,71,V,68.0,2,39847.0,6,RED,187,180,-0.706546,-0.765855,Rejected,Z1,Z1,"(Z1, Z1)"
4,4,de538100bb0f4137961b9551640ef35e,3,Fri,1182,R,1183.0,2,34912.0,3,GREEN,972,922,1.014233,0.469273,Accepted,Z9,Z9,"(Z9, Z9)"


In [20]:
data = data[data['lead_days'] >= 0]

In [21]:
data['order_distance_norm'] = (data['order_distance'] - data['order_distance'].mean())/(data['order_distance'].std())
data['weight_norm'] = (data['order_origin_weight'] - data['order_origin_weight'].mean())/(data['order_origin_weight'].std())
data['lead_days_norm'] = (data['lead_days'] - data['lead_days'].mean())/(data['lead_days'].std())
data['week_id_norm'] = (data['week_id'] - data['week_id'].mean())/(data['week_id'].std())

In [22]:
coords = {
    "obs_id": np.arange((data.shape)[0]),
    #"zipzone_pair": zipzone_pairs,
    "num_coef": np.arange(5)
}

In [23]:
order_distance_norm = data.order_distance_norm
weight_norm = data.weight_norm
lead_days_norm = data.lead_days_norm
week_id_norm = data.week_id_norm
rate_norm = data.rate_norm
est_cost_norm = data.est_cost_norm

order_distance_norm = np.array(order_distance_norm.values.tolist())
weight_norm = np.array(weight_norm.values.tolist())
lead_days_norm = np.array(lead_days_norm.values.tolist())
week_id_norm = np.array(week_id_norm.values.tolist())
rate_norm = np.array(rate_norm.values.tolist())
est_cost_norm = np.array(est_cost_norm.tolist())

## Pooled Rate Model

In [26]:
with pm.Model(coords=coords) as rate_model_1:
    #zipzone_pair_idx = pm.Data("zipzone_pair_idx", zipzone_pair_id, dims = 'obs_id')
    x1_shared = pm.Data("distance", order_distance_norm, dims = 'obs_id')
    x2_shared = pm.Data("weight", weight_norm, dims = 'obs_id')
    x3_shared = pm.Data("leaddays", lead_days_norm, dims = 'obs_id')
    x4_shared = pm.Data("weekid", week_id_norm, dims = 'obs_id')
    y_shared = pm.Data("rate", rate_norm, dims = 'obs_id')
    
    
    sd_dist = pm.Exponential.dist(1.0)
    
    
    chol, corr, stds = pm.LKJCholeskyCov("chol", n=5, eta=2.0, sd_dist=sd_dist, compute_corr=True)
    
    
    mu_intercept = pm.Normal('mu_intercept', mu=0, sigma=1)
    mu_distance = pm.Normal('mu_distance', mu=0, sigma=1)
    mu_weight = pm.Normal('mu_weight', mu=0, sigma=1)
    mu_leaddays = pm.Normal('mu_leaddays', mu=0, sigma=1)
    mu_weekid = pm.Normal('mu_weekid', mu=0, sigma=1)
    
    
    cov_coef = pm.MvNormal("cov_coef", mu=tt.stack([mu_intercept, mu_distance, mu_weight, mu_leaddays, mu_weekid]), 
                           chol=chol, dims = 'num_coef')
    
    
    # Model error
    epsilon = pm.HalfNormal('epsilon', 1) # Possibly change distribution
    
    # Regression for mean of likelihood
    y_hat = cov_coef[0] + cov_coef[1] * x1_shared + cov_coef[2] * x2_shared + cov_coef[3] * x3_shared + cov_coef[4] * x4_shared

    
    # Likelihood
    pm.StudentT('y_like', nu=5, mu=y_hat, sigma=epsilon, observed=y_shared, dims = 'obs_id')
    
    
    rate_trace_1 = pm.sample(1000, tune = 1000, cores=2, return_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [epsilon, cov_coef, mu_weekid, mu_leaddays, mu_weight, mu_distance, mu_intercept, chol]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 2120 seconds.


MemoryError: Unable to allocate 1.15 GiB for an array with shape (1000, 154929) and data type float64

In [ ]:
with rate_model_1:
    rate_post_pred_11 = pm.sample_posterior_predictive(rate_trace_1, var_names = ['y_like'])

In [ ]:
az.plot_ppc(az.from_pymc3(posterior_predictive=rate_post_pred_11, model=rate_model_1))

In [ ]:
summary = az.summary(rate_trace_1, round_to=8)
summary

In [ ]:
from pymc3 import traceplot

traceplot(rate_trace_1)

### Use this model to predict rate_norm on just Z3->Z3 data

In [ ]:
Z3_data = data[(data['ZipZone_origin'] == 'Z3') & (data['ZipZone_dest'] == 'Z3')]

Z3_distance = Z3_data.order_distance_norm
Z3_distance = np.array(Z3_distance.values.tolist())

Z3_weight = Z3_data.weight_norm
Z3_weight = np.array(Z3_weight.values.tolist())

Z3_leaddays = Z3_data.lead_days_norm
Z3_leaddays = np.array(Z3_leaddays.values.tolist())

Z3_weekid = Z3_data.week_id_norm
Z3_weekid = np.array(Z3_weekid.values.tolist())

In [ ]:
with rate_model_1:
    # change the value and shape of the data
    pm.set_data(
        {
            #"zipzone_pair_idx": np.tile(zipzone_pair_id[8], (data.shape)[0]),
            "distance": np.concatenate((Z3_distance, np.empty((data.shape)[0] - len(Z3_distance)))),
            "weight": np.concatenate((Z3_weight, np.empty((data.shape)[0] - len(Z3_weight)))),
            "leaddays": np.concatenate((Z3_leaddays, np.empty((data.shape)[0] - len(Z3_leaddays)))),
            "weekid": np.concatenate((Z3_weekid, np.empty((data.shape)[0] - len(Z3_weekid)))),
            # use dummy values with the same shape:
            "rate": np.empty(data.shape[0]),
        }
    )
    rate_post_pred_12 = pm.sample_posterior_predictive(rate_trace_1, var_names=['y_like'])
    #az.from_pymc3_predictions(rate_post_pred_12, idata_orig=rate_trace_1, inplace=True, coords=prediction_coords)

In [ ]:
#az.plot_posterior(rate_trace_1);

In [ ]:
# Generate the actual prediction values
p_test_pred = rate_post_pred_12["y_like"].mean(axis=0)

## Pooled Cost Model

In [ ]:
with pm.Model(coords=coords) as cst_model_1:
    #zipzone_pair_idx = pm.Data("zipzone_pair_idx", zipzone_pair_id, dims = 'obs_id')
    x1_shared = pm.Data("distance", order_distance_norm, dims = 'obs_id')
    x2_shared = pm.Data("weight", weight_norm, dims = 'obs_id')
    x3_shared = pm.Data("leaddays", lead_days_norm, dims = 'obs_id')
    x4_shared = pm.Data("weekid", week_id_norm, dims = 'obs_id')
    y_shared = pm.Data("cost", est_cost_norm, dims = 'obs_id')
    
    
    sd_dist = pm.Exponential.dist(1.0)
    
    
    chol, corr, stds = pm.LKJCholeskyCov("chol", n=5, eta=2.0, sd_dist=sd_dist, compute_corr=True)
    
    
    mu_intercept = pm.Normal('mu_intercept', mu=0, sigma=1)
    mu_distance = pm.Normal('mu_distance', mu=0, sigma=1)
    mu_weight = pm.Normal('mu_weight', mu=0, sigma=1)
    mu_leaddays = pm.Normal('mu_leaddays', mu=0, sigma=1)
    mu_weekid = pm.Normal('mu_weekid', mu=0, sigma=1)
    
    
    cov_coef = pm.MvNormal("cov_coef", mu=tt.stack([mu_intercept, mu_distance, mu_weight, mu_leaddays, mu_weekid]), 
                           chol=chol, dims = 'num_coef')
    
    
    # Model error
    epsilon = pm.HalfNormal('epsilon', 1) # Possibly change distribution
    
    # Regression for mean of likelihood
    y_hat = cov_coef[0] + cov_coef[1] * x1_shared + cov_coef[2] * x2_shared + cov_coef[3] * x3_shared + cov_coef[4] * x4_shared

    
    # Likelihood
    pm.StudentT('y_like', nu=5, mu=y_hat, sigma=epsilon, observed=y_shared, dims = 'obs_id')
    
    
    cost_trace_1 = pm.sample(1000, tune = 1000, cores=2, return_inferencedata=True)

In [ ]:
with cost_model_1:
    cost_post_pred_11 = pm.sample_posterior_predictive(cost_trace_1, var_names = ['y_like'])

In [ ]:
az.plot_ppc(az.from_pymc3(posterior_predictive=cost_post_pred_11, model=cost_model_1))

In [ ]:
summary = az.summary(cost_trace_1, round_to=8)
summary

In [ ]:
from pymc3 import traceplot

traceplot(cost_trace_1)

### Use this model to predict rate_norm on just Z3->Z3 data

In [ ]:
with cost_model_1:
    # change the value and shape of the data
    pm.set_data(
        {
            #"zipzone_pair_idx": np.tile(zipzone_pair_id[8], (data.shape)[0]),
            "distance": np.concatenate((Z3_distance, np.empty((data.shape)[0] - len(Z3_distance)))),
            "weight": np.concatenate((Z3_weight, np.empty((data.shape)[0] - len(Z3_weight)))),
            "leaddays": np.concatenate((Z3_leaddays, np.empty((data.shape)[0] - len(Z3_leaddays)))),
            "weekid": np.concatenate((Z3_weekid, np.empty((data.shape)[0] - len(Z3_weekid)))),
            # use dummy values with the same shape:
            "cost": np.empty(data.shape[0]),
        }
    )
    cost_post_pred_12 = pm.sample_posterior_predictive(cost_trace_1, var_names=['y_like'])
    #az.from_pymc3_predictions(rate_post_pred_12, idata_orig=rate_trace_1, inplace=True, coords=prediction_coords)

In [ ]:
# Generate the actual prediction values
p_test_pred = cost_post_pred_12["y_like"].mean(axis=0)